In [104]:
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd

import credentials

In [5]:
conn_str = 'mysql+mysqlconnector://' + \
	f"{credentials.mysql['username']}:{credentials.mysql['password']}" + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

In [6]:
ids = pd.read_sql_table("Identifications", connection)

/home/nelson/anaconda3/lib/python3.13/site-packages/pandas/io/sql.py:1725: SAWarning: Did not recognize type 'polygon' of column 'Shape'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)


In [7]:
noiddate = pd.read_sql(
	"select IdentificationID,DateEnd from Identifications left join Occurrences on Occurrence=OccurrenceID where Date is NULL",
	connection)

In [8]:
ids = ids.merge(noiddate, how='left', on='IdentificationID')

In [9]:
ids.loc[ids.Date.isnull(), 'Date'] = pd.to_datetime(ids.loc[ids.Date.isnull(), 'DateEnd'])

In [ ]:
# Debugging selecting last ided occurrences
#dt = pd.DataFrame.from_dict({
#	'd':['2014-10-16', '2014-10-18', '2014-10-16', '2016-10-16', None, None],
#	'v':[1,1,2,2,2,3],
#	'a':[3,4,4,3,3,4],
#	'i': [1,2,3,4,5,6]})
#dt['d'] = pd.to_datetime(dt.d)
#l = dt.groupby(['v']).d.idxmax()
#onlynas = dt.loc[dt.v.isin(l[l.isnull()].index)].groupby('v').first().reset_index()[dt.columns] # Only NaT dates
#pd.concat([onlynas, dt.loc[l[l.notna()]]])

,d,v,a,i
0,NaT,3,4,6
1,2014-10-18,1,4,2
3,2016-10-16,2,3,4


In [10]:
lastids = ids.groupby('Occurrence').Date.idxmax()

In [11]:
onlynas = ids.loc[
		ids.Occurrence.isin(lastids[lastids.isnull()].index)
	].groupby('Occurrence'
	).first(
	).reset_index(
	)[ids.columns]

In [12]:
id_pool = pd.concat([onlynas, ids.loc[lastids[lastids.notnull()]]]).IdentificationID

In [13]:
tax = pd.read_sql_table("Taxa", connection)

In [14]:
clades = [
	'Polypodiidae',
	'Lycopodiopsida',
	'Ophioglossidae',
	'Equisetidae',
	'Marattiidae'
]

In [15]:
target = []
papa = tax.loc[tax.Name.isin(clades), 'TaxonID'].tolist()

while papa:
	target += papa
	papa = tax.loc[tax.Parent.isin(papa), 'TaxonID'].tolist()
	#print(papa)

In [43]:
pooltax = ', '.join([str(i) for i in target])

In [49]:
tax[tax.TaxonID.isin(target)].head()

,TaxonID,Name,AcceptedName,Authority,Author,Protologue,Parent,Comment,CheckPriority,Distribution,TimeStamp
65,66,Acrostichum aureum,None,None,L.,None,4002.0,None,NaN,None,2025-06-18 09:14:24
66,67,Adiantum,None,None,None,None,2934.0,None,NaN,None,2025-06-18 09:14:24
67,68,Adiantum concinnum,None,None,Humb. & Bonpl. ex Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
68,69,Adiantum patens,None,None,Willd.,None,67.0,None,NaN,None,2025-06-18 09:14:24
69,70,Adiantum petiolatum,None,None,Desv.,None,67.0,None,NaN,None,2025-06-18 09:14:24


In [17]:
poollist = ', '.join([str(x) for x in id_pool.tolist()])

In [116]:
recs = pd.read_sql(
	"SELECT Specimens.SpecimenCode AS 'catalogNumber', Occurrences.Type AS 'basisOfRecord', Specimens.Institution AS 'institutionCode', Occurrences.CollectorVerbatim AS 'recordedBy', Occurrences.CollectionNumberVerbatim AS 'recordNumber', Occurrences.DateInit AS 'eventDate', Taxa.Name AS 'scientificName', Locations.Country AS 'country', Locations.Admin01 AS 'stateProvince', Locations.Admin02 AS 'municipality', Locations.Admin03 as 'county', Locations.Name AS 'localityVerbatim', Geocodings.InterpretedLat AS 'decimalLatitude', Geocodings.InterpretedLon AS 'decimalLongitude', Locations.ElevationMin AS 'minimumElevationInMeters', Locations.ElevationMax AS 'maximumElevationInMeters' FROM Occurrences " \
	+ "LEFT JOIN Identifications ON Identifications.Occurrence=OccurrenceID " \
	+ "LEFT JOIN Taxa ON TaxonID=Identifications.Name " \
	+ "LEFT JOIN Specimens ON Specimens.Occurrence=OccurrenceID " \
	+ "LEFT JOIN Locations ON LocationID=Occurrences.Location " \
	+ "LEFT JOIN Geocodings ON LocationID=Geocodings.Location " \
	+ f"WHERE Identifications.IdentificationID IN ({poollist}) " \
	+ f"AND Taxa.TaxonID IN ({pooltax})",
	connection)

In [117]:
recs['taxonRank'] = None

In [120]:
if recs.loc[recs.scientificName.str.contains('ales$', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('ales$', na=False),
		'taxonRank'] = 'order'

In [121]:
if recs.loc[recs.scientificName.str.contains('aceae$', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('aceae$', na=False),
		'taxonRank'] = 'family'

In [122]:
if recs.loc[recs.scientificName.str.contains('×', na=False)
	].shape[0] > 0:

	recs.loc[
		recs.scientificName.str.contains('×', na=False),
		'taxonRank'] = 'hybrid'

In [123]:
if recs.loc[
	~recs.scientificName.str.contains('×', na=False) &
	recs.scientificName.str.contains('\w\s+\w', na=False)
	].shape[0] > 0:

	recs.loc[
		~recs.scientificName.str.contains('×', na=False) &
		recs.scientificName.str.contains('\w\s+\w', na=False),
		'taxonRank'] = 'species'

In [ ]:
if recs.loc[recs.taxonRank.isnull()].shape[0] > 0:
	recs.loc[recs.taxonRank.isnull(), 'taxonRank'] = 'genus'

asd


In [55]:
connection.close()